In [80]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
#from esda.moran import Moran
from libpysal.weights.contiguity import Queen
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [81]:
#path1 = r"C:\Users\PcLaptop\Documents\GitHub\Climate-and-conflict\Datasets\som_adm_ocha_itos_20230308_shp\som_admbnda_adm1_ocha_20230308.shp"
path2 = r"/home/sara/Documenti/GitHub/Climate-and-conflict/Datasets/som_adm_ocha_itos_20230308_shp/som_admbnda_adm1_ocha_20230308.shp"
states_gdf = gpd.read_file(path2)

#file1 = r"C:\Users\PcLaptop\Dropbox (Politecnico Di Torino Studenti)\Untitled Folder\distance_df.csv"
file2 = r"/home/sara/Documenti/GitHub/Climate-and-conflict/Datasets/ACLED_1997-01-01-2023-07-18_Somalia.csv"
dist = pd.read_csv(file2)

#csv1 = r"C:\Users\PcLaptop\Documents\GitHub\Climate-and-conflict\df_with_dummies.csv"
csv2 = r"/home/sara/Documenti/GitHub/Climate-and-conflict/csv/df_lag1_disp.csv"
df_with_dummies = pd.read_csv(csv2)

asc = r"/home/sara/Documenti/GitHub/Climate-and-conflict/Datasets/gpw_v4_admin_unit_center_points_population_estimates_rev11_som.csv"
population_density = pd.read_csv(asc)

In [82]:
population_density=population_density.groupby('NAME2')[['UN_2000_DS','UN_2005_DS','UN_2010_DS','UN_2015_DS','UN_2020_DS']].sum()
population_density.rename(index={'Juba Dhexe (Middle)':'Middle_Juba','Juba Hoose (Lower)':'Lower_Juba','Shabelle Dhexe (Middle)':'Middle_Shabelle','Shabelle Hoose (Lower)':'Lower_Shabelle','Woqooyi Galbeed':'Woqooyi_Galbeed'},inplace=True)	

In [83]:
population_density['UN_2001_DS']=population_density['UN_2000_DS']+(population_density['UN_2005_DS']-population_density['UN_2000_DS'])/5
population_density['UN_2002_DS']=population_density['UN_2001_DS']+(population_density['UN_2005_DS']-population_density['UN_2000_DS'])/5
population_density['UN_2003_DS']=population_density['UN_2002_DS']+(population_density['UN_2005_DS']-population_density['UN_2000_DS'])/5
population_density['UN_2004_DS']=population_density['UN_2003_DS']+(population_density['UN_2005_DS']-population_density['UN_2000_DS'])/5
population_density['UN_2006_DS']=population_density['UN_2005_DS']+(population_density['UN_2010_DS']-population_density['UN_2005_DS'])/5
population_density['UN_2007_DS']=population_density['UN_2006_DS']+(population_density['UN_2010_DS']-population_density['UN_2005_DS'])/5
population_density['UN_2008_DS']=population_density['UN_2007_DS']+(population_density['UN_2010_DS']-population_density['UN_2005_DS'])/5
population_density['UN_2009_DS']=population_density['UN_2008_DS']+(population_density['UN_2010_DS']-population_density['UN_2005_DS'])/5
population_density['UN_2011_DS']=population_density['UN_2010_DS']+(population_density['UN_2015_DS']-population_density['UN_2010_DS'])/5
population_density['UN_2012_DS']=population_density['UN_2011_DS']+(population_density['UN_2015_DS']-population_density['UN_2010_DS'])/5
population_density['UN_2013_DS']=population_density['UN_2012_DS']+(population_density['UN_2015_DS']-population_density['UN_2010_DS'])/5
population_density['UN_2014_DS']=population_density['UN_2013_DS']+(population_density['UN_2015_DS']-population_density['UN_2010_DS'])/5
population_density['UN_2016_DS']=population_density['UN_2015_DS']+(population_density['UN_2020_DS']-population_density['UN_2015_DS'])/5
population_density['UN_2017_DS']=population_density['UN_2016_DS']+(population_density['UN_2020_DS']-population_density['UN_2015_DS'])/5
population_density['UN_2018_DS']=population_density['UN_2017_DS']+(population_density['UN_2020_DS']-population_density['UN_2015_DS'])/5
population_density['UN_2019_DS']=population_density['UN_2018_DS']+(population_density['UN_2020_DS']-population_density['UN_2015_DS'])/5
population_density['UN_2021_DS']=population_density['UN_2020_DS']+(population_density['UN_2020_DS']-population_density['UN_2015_DS'])/5
population_density['UN_2022_DS']=population_density['UN_2021_DS']+(population_density['UN_2020_DS']-population_density['UN_2015_DS'])/5

population_density=population_density[['UN_2000_DS','UN_2001_DS','UN_2002_DS','UN_2003_DS','UN_2004_DS','UN_2005_DS','UN_2006_DS','UN_2007_DS','UN_2008_DS','UN_2009_DS','UN_2010_DS','UN_2011_DS','UN_2012_DS','UN_2013_DS','UN_2014_DS','UN_2015_DS','UN_2016_DS','UN_2017_DS','UN_2018_DS','UN_2019_DS','UN_2020_DS','UN_2021_DS','UN_2022_DS']]
population_density.reset_index(inplace=True)
population_density.rename(columns={'NAME2':'admin1','UN_2000_DS':'2000','UN_2001_DS':'2001','UN_2002_DS':'2002','UN_2003_DS':'2003','UN_2004_DS':'2004','UN_2005_DS':'2005','UN_2006_DS':'2006','UN_2007_DS':'2007','UN_2008_DS':'2008','UN_2009_DS':'2009','UN_2010_DS':'2010','UN_2011_DS':'2011','UN_2012_DS':'2012','UN_2013_DS':'2013','UN_2014_DS':'2014','UN_2015_DS':'2015','UN_2016_DS':'2016','UN_2017_DS':'2017','UN_2018_DS':'2018','UN_2019_DS':'2019','UN_2020_DS':'2020','UN_2021_DS':'2021','UN_2022_DS':'2022'},inplace=True)

In [84]:
#pivot the table to have the years as rows
population_density=population_density.melt(id_vars=['admin1'],value_vars=['2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022'],var_name='year',value_name='population_density')
population_density['year']=population_density['year'].astype(int)
#merge the df_with_dummies with the population density for each region for each year
df_with_dummies=pd.merge(df_with_dummies,population_density,how='left',on=['year','admin1'])

In [85]:
y_var_name = 'conflicts'
X_var_names = ['TA_lag1','PA_lag1','DL_lag1', 'population_density', 'sum_disp']

In [86]:
# Regression expression for OLS with dummies

unit_names = df_with_dummies['admin1'].unique().tolist()
unit_names.sort()
unit_names_t = df_with_dummies['month_name'].unique().tolist()
unit_names_mr = (df_with_dummies['admin1'] + df_with_dummies['month_name']).unique().tolist()

lsdv_expr = y_var_name + ' ~ '
i = 0
for X_var_name in X_var_names:
    if i > 0:
        lsdv_expr = lsdv_expr + ' + ' + X_var_name
    else:
        lsdv_expr = lsdv_expr + X_var_name
    i = i + 1
for dummy_name in unit_names[:-1]:
   lsdv_expr = lsdv_expr + ' + ' + dummy_name + '_x'
for dummy_name_t in unit_names_t[:-1]:
    lsdv_expr = lsdv_expr + ' + ' + dummy_name_t
#for dummy_name_mr in unit_names_mr[:-1]:
   # lsdv_expr = lsdv_expr + ' + ' + dummy_name_mr
#lsdv_expr=lsdv_expr + ' - 1' 
print('Regression expression for OLS with dummies=' + lsdv_expr)

Regression expression for OLS with dummies=conflicts ~ TA_lag1 + PA_lag1 + DL_lag1 + population_density + sum_disp + Awdal_x + Bakool_x + Banadir_x + Bari_x + Bay_x + Galgaduud_x + Gedo_x + Hiraan_x + Lower_Juba_x + Lower_Shabelle_x + Middle_Juba_x + Middle_Shabelle_x + Mudug_x + Nugaal_x + Sanaag_x + Sool_x + Togdheer_x + January + February + March + April + May + June + July + August + September + October + November


In [87]:
lsdv_model = smf.ols(formula=lsdv_expr, data=df_with_dummies)
lsdv_model_results = lsdv_model.fit()
print(lsdv_model_results.summary())

                            OLS Regression Results                            
Dep. Variable:              conflicts   R-squared:                       0.824
Model:                            OLS   Adj. R-squared:                  0.820
Method:                 Least Squares   F-statistic:                     210.1
Date:                Tue, 03 Oct 2023   Prob (F-statistic):               0.00
Time:                        15:30:28   Log-Likelihood:                -5134.5
No. Observations:                1512   AIC:                         1.034e+04
Df Residuals:                    1478   BIC:                         1.052e+04
Df Model:                          33                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              3.3890      1

In [88]:
df_with_dummies['conflicts_pro_capite']=df_with_dummies['conflicts']/df_with_dummies['population_density']

In [89]:
sp = r"/home/sara/Documenti/GitHub/Climate-and-conflict/drought/spei_somalia.csv"
spei = pd.read_csv(sp)

In [90]:
spei['time']=spei['time'].astype(str).str[:7]

In [91]:
#in  column admin1 substitute the name of the regions  'Middle Juba'  with 'Middle_Juba'
spei['admin1']=spei['admin1'].replace('Middle Juba','Middle_Juba')
spei['admin1']=spei['admin1'].replace('Lower Juba','Lower_Juba')
spei['admin1']=spei['admin1'].replace('Middle Shabelle','Middle_Shabelle')
spei['admin1']=spei['admin1'].replace('Lower Shabelle','Lower_Shabelle')
spei['admin1']=spei['admin1'].replace('Woqooyi Galbeed','Woqooyi_Galbeed')


In [92]:
#merge the df_with_dummies with the spei for each region for each year
df_tot=pd.merge(df_with_dummies,spei,how='left',on=['time','admin1'])

In [93]:
# Add Banadir region with tmx and pre as mean of the neighbouring regions

district1 = 'Lower_Shabelle'  
district2 = 'Middle_Shabelle'  

# Calculate the mean tmx and pre for the neighboring districts
sp = df_tot[(df_tot['admin1']==district1) | (df_tot['admin1']==district2)].groupby('time')['spei'].mean()

new_data = pd.DataFrame({ 'admin1': 'Banadir', 'spei': sp}).reset_index()

# Append the new DataFrame to the original DataFrame
df3 = pd.merge(df_tot, new_data, on=['admin1', 'time'], how='outer')

In [94]:
#merge spei_x and spei_y in one column
df3['spei_x'].fillna(df3['spei_y'], inplace=True)
#drop spei_x and spei_y
df3=df3.drop(['spei_y'],axis=1)
df3.rename(columns={'spei_x':'spei'},inplace=True)

In [95]:
df3

,index,time,admin1,year,month,TA,PA,DL,TA_lag1,PA_lag1,...,Mudug_y,Nugaal_y,Sanaag_y,Sool_y,Togdheer_y,Woqooyi_Galbeed_y,sum_disp,population_density,conflicts_pro_capite,spei
0,1380,2016-01,Awdal,2016,1,1.928297,-0.380389,91,2.434686,-0.327348,...,0.0,6.0,0.0,0.0,0.0,0.0,89.0,141.666530,0.028235,-0.328641
1,1380,2016-01,Bakool,2016,1,1.763821,0.482869,91,1.788443,0.451277,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,103.529746,0.057954,-1.283446
2,1380,2016-01,Banadir,2016,1,0.945572,0.199790,12,0.863306,0.142254,...,0.0,0.0,0.0,0.0,0.0,0.0,7018.0,6288.475825,0.005725,-0.972502
3,1380,2016-01,Bari,2016,1,2.795625,0.105978,22,3.176745,0.039772,...,0.0,16.0,0.0,0.0,0.0,0.0,75.0,62.337389,0.096250,-2.106666
4,1380,2016-01,Bay,2016,1,1.215753,0.388470,91,1.161052,0.406663,...,0.0,0.0,0.0,0.0,0.0,0.0,184.0,99.131867,0.161401,-1.476367
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1507,1463,2022-12,Nugaal,2022,12,0.242491,0.045570,2,0.000447,0.236927,...,0.0,1552.0,0.0,180.0,0.0,0.0,1732.0,23.889637,0.041859,NaN
1508,1463,2022-12,Sanaag,2022,12,0.890553,0.055662,17,0.499523,0.099197,...,0.0,0.0,493.0,0.0,0.0,0.0,493.0,27.283836,0.000000,NaN
1509,1463,2022-12,Sool,2022,12,0.717894,0.046859,17,0.367086,0.187917,...,170.0,0.0,0.0,873.0,90.0,0.0,1133.0,26.175893,0.305625,NaN
1510,1463,2022-12,Togdheer,2022,12,1.239002,-0.014982,17,0.894558,0.003856,...,142.0,0.0,0.0,203.0,490.0,0.0,835.0,69.219380,0.000000,NaN


In [98]:
#df3.to_csv(r'/home/sara/Documenti/GitHub/Climate-and-conflict/csv/df_lag1_disp_pop_spei.csv',index=False)